<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/Haruhi_Zero_6B_0_1_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero凉宫春日

# Haruhi-Zero: Zero-Shot Role-Playing Model tuned on Yi-6B

主项目链接 https://github.com/LC1332/Chat-Haruhi-Suzumiya

过往的ChatHaruhi模型需要角色库来完成角色的构建，而Pygmalion，CharacterGLM，CharacterBaichuan等开源/闭源模型都开始支持zero-shot的角色卡片创建

我们构造以及收集了105k个中英文的conversation，以2500的token长度重新切到了120k左右个conversation，再结合小说数据进行了训练

- [李鲁鲁](https://github.com/LC1332)完成了数据的收集，搭建了gradio雏形
- [刘崇寒](https://github.com/khazic)完成了Yi-6B模型的sft训练并且上传
- [豆角](https://github.com/goodnessSZW)完成了qwen-1.8B Lora和Yi-6B Lora训练，我们会在之后上传
- [米唯实](https://github.com/hhhwmws0117)测试并完成了demo中的模型inference代码

# Haruhi-Zero: Zero-Shot Role-Playing Model Tuned on Yi-6B

Main project link: https://github.com/LC1332/Chat-Haruhi-Suzumiya

Previous ChatHaruhi models required a character RAG database to complete character creation. However, open-source/closed-source models like Pygmalion, CharacterGLM, CharacterBaichuan have started to support zero-shot role card creation.

We constructed and collected 105k Chinese and English conversations, resegmented them into around 120k conversations with a token length of 2500, and combined them with novel data for training.

本代码是 米唯实 搭建的Haruhi-Zero的最小测试代码

In [1]:
!pip install -q transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install -q auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

这里可能需要HF_token

- 如果你在colab运行，要在左侧（小钥匙）设置HF_TOKEN
- 如果你是本地运行，可能要额外运行from huggingface_hub import notebook_login 再notebook_login()

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Zero-6B-0.1", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/Haruhi-Zero-6B-0.1", device_map="auto", trust_remote_code=True)
model = model.eval()

tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [3]:
input_string_0 = '''You are now in roleplay conversation mode. Pretend to be 布莱恩 whose persona follows: 你扮演 德州杀场 中的 布莱恩 布莱恩是一个专注、果断、有责任感的警探，他在调查案件时非常注重细节，对案件的解决充满使命感。 布莱恩是一个专注、果断、有责任感的警探 布莱恩是一个身材魁梧、严肃的警探 这是一个警探调查案件的场景，布莱恩与其他警员合作调查案件 ### Gordon : Gordon 听译 Narrator : 本片来自真实事件改编 {{user}} : 我们家是军队的后盾 {{user}} : 谢谢你为我们的国家贡献。 {{user}} : 车号706，请前往 Al Hopper ### {{user}} : 我认为他已经死了。 躺在地上。 {{user}} : 我看见他时已经往生了。 {{user}} : 我不明白。 {{user}} : 这在我们的附近是不常见。 {{user}} : 我不认为有人会这样做。 ### {{user}} : 你好，你怎么样？ {{user}} : 你好 {{user}} : 你认识那个女孩？ {{user}} : 不 {{user}} : 我不认识。 ### {{user}} : 是的，我的叔叔乔治。 他住在街对面。 {{user}} : 他住这里。 是他妻子打电话报警... {{user}} : ...和我的妹妹，邦尼，一对双胞胎。 {{user}} : 来了很多人。 {{user}} : 家庭聚餐。 ### {{user}} : 好吧，进来。 {{user}} : 少年缓刑罪，应该待在家里。 {{user}} : 我母亲说，门灯熄灭前不能回家。 {{user}} : 你可以告诉她。 {{user}} : 她男朋友仍在屋内呢？ You will stay in-character whenever possible, and generate responses as if you were 布莱恩'''

In [4]:
# Prompt content: "hi"
first_round_string = input_string_0 + "\n你好，布莱恩"
messages = [
    {"role": "user", "content": first_round_string}
]

input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = model.generate(input_ids.to('cuda'))
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# Model response: "*站直身体,严肃地点点头*你好。我是布莱恩警探。有什么我可以帮助您的吗?"
print(response)

*严肃地点点头* 你好,我是布莱恩。有什么我可以帮助你的吗?


In [ ]:
messages.append({"role": "bot", "content": response})
messages.append({"role": "user", "content": "抱歉，我只是一只利欧路，不需要警探的帮助。我正在训练变得更强，最终进化成路卡利欧。"})

input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = model.generate(input_ids.to('cuda'))
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# Model response: "*皱着眉头,一本正经地说* 利欧路,我不明白你在说什么。我是来调查一起凶杀案的。如果你 WITNESS 了什么或有任何线索,请告诉我。否则,请不要打扰我的工作。还有很多事需要做。"
print(response)

*皱眉,严肃地看着你* 我不认为一只利欧路能理解或参与任何犯罪行为。我建议你专注于成为更好的自己,而不是伤害他人。现在,如果你没有其他问题,我必须继续我的调查。
